In [ ]:
#https://keras.io/examples/generative/vae/
#https://keras.io/examples/generative/molecule_generation/

In [ ]:
# !python3 -m pip install --upgrade pip
# !pip install pydot
# !apt-get install -y graphviz

In [ ]:
try creatmodel without volumns

In [13]:
import os,shutil,random
from sklearn.preprocessing import MinMaxScaler
import joblib
from IPython.display import clear_output
import pandas as pd
import h5py
import tensorflow as tf
from IPython.display import clear_output,display, HTML
from sklearn.model_selection import train_test_split
import datetime as dt
import numpy as np
from src.model import Sampling
#================== initialization ==================
currentTM=dt.datetime.now().strftime("%Y-%m-%dT%H%M%S")
PROJECT = "testVAEModel"
LATENT_DIM = 32
VAE_LR = 0.001
EPOCHS = 10
BATCH_SIZE = 32

PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['DTE','INTRINSIC_VALUE', 'TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
MODEL_PATH = "./models/"
H5_PATH = './data/OptTrainData/'
DISPLAY = False
WANDB_LOG = True
RESUME = False

Scaler = joblib.load(SCALER_PATH )

In [2]:
notes = f"""
test Run use_bias set false , no tranfrom
"""
import wandb
from wandb.keras import WandbCallback
CONFIG = {"latent_dim":LATENT_DIM,
          "learning_rate": VAE_LR,
          "epochs": EPOCHS,
          "batch_size": BATCH_SIZE,
          "architecture": "VAE",
          "dataset": "OptionsChaine",
          "encoder_dense_units":[128,64],
          "encoder_dropout_rate":0.2,
          "decoder_dense_units":[64, 128],
          "decoder_dropout_rate":0.2,
          "use_bias":False,
          "transform":True
           }

if WANDB_LOG :
    wandb.login()
    run = wandb.init(project=PROJECT, 
                     name=currentTM, 
                     config=CONFIG,
                     notes=notes
                    )

wandb: Currently logged in as: wasan-sinlapa. Use `wandb login --relogin` to force relogin


In [ ]:
#Example

# from IPython.display import clear_output,display, HTML
# import numpy as np
# #load scaler
# scaler = MinMaxScaler()
# PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
# random.shuffle(PartitionDate)
# scaler = joblib.load(SCALER_PATH)


# for i,partdate in enumerate(PartitionDate) :
#     df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
#                                  , filters=[('PartitionDate', '=', partdate)]
#                                 )
#     df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
#     df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
#     DATA  = np.empty((0,) + (20,9) ) 
#     for opt_id in np.unique( df[["OPTIONS_ID"]].values):
#         df_filter  = df[df["OPTIONS_ID"]==opt_id]
#         if len(df_filter) == 20:
#             DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))
#         else:
#             #print( len(df_filter) )
#             #display(HTML(df_filter[['STRIKE']+SCALER_COL].to_html()))
#             pass
            
#     ## Save the NumPy array to an HDF5 file
#     # with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
#     #     dset = f.create_dataset(f'{partdate}', data=DATA, chunks=True , compression='gzip')

#     print(f"[Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')

In [ ]:
# # Save the NumPy array to an HDF5 file
# with h5py.File(H5_PATH, 'w') as f:
#     #dset = f.create_dataset('dataset', data=DATA, chunks=True, compression='gzip')
#     #test
#     dset = f.create_dataset('dataset', data=DATA, compression='gzip')

In [ ]:
#=====================================================================

In [3]:
from src.model import OptionChainGenerator
from src.layer import encoder, decoder

model = OptionChainGenerator(
    encoder(latent_dim = LATENT_DIM, 
            input_shape= (32,3), 
            dense_units = CONFIG["encoder_dense_units"], 
            dropout_rate= CONFIG["encoder_dropout_rate"],
            use_bias=CONFIG["use_bias"]
           ), 
    decoder(latent_dim  = LATENT_DIM , 
            output_shape= (32,1),
            dense_units = CONFIG["decoder_dense_units"],
            dropout_rate= CONFIG["decoder_dropout_rate"],
            use_bias=CONFIG["use_bias"]
           )
)

def dummy_loss(y_true, y_pred):
    return 0.0
    
vae_optimizer = tf.keras.optimizers.Adam(learning_rate=VAE_LR)
model.compile(vae_optimizer )#, loss=dummy_loss)

In [ ]:
################## show model ######################
if DISPLAY :
    from tensorflow.keras.utils import model_to_dot
    from IPython.display import SVG, display
    
    def display_model(model, width=1024, height=512):
        dot = model_to_dot(model, show_shapes=True, show_layer_names=True)
        svg_data = dot.create(prog='dot', format='svg').decode("utf-8")
        svg_html = f'<div style="width:{width}px;height:{height}px;">{svg_data}</div>'
        display(HTML(svg_html))

In [ ]:
## Example usage:
## Display the encoder model with reduced size
if DISPLAY :
    display_model(model.encoder, width=1024, height=512)

In [ ]:
if DISPLAY :
    display_model(model.decoder, width=2500, height=512)

In [ ]:
#================== loadmodel ====================

In [4]:
from tensorflow.keras.models import load_model
model_path = MODEL_PATH+f'{PROJECT}'
if not RESUME :
    if os.path.exists(model_path) :
        shutil.rmtree(model_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)
    model.encoder.save(model_path+f'/'+f'encoder.keras') 
    model.decoder.save(model_path+f'/'+f'decoder.keras') 
else:
    model.encoder = load_model(model_path+'/'+f'encoder.keras') 
    model.decoder = load_model(model_path+'/'+f'decoder.keras') 

In [ ]:
#================== train model ==================
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)

STOP_MODEL = False
STACK_DATA = np.empty((0,) + (32,9) ) #init STACK_DATA
for partdate in PartitionDate[:] :
    clear_output(wait=False)
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    ###transform
    if CONFIG['transform'] :
        DATA = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
        DATA = DATA.reshape(data_shape)
    DATA = np.vstack((DATA ,STACK_DATA))
    if len(DATA) < 64 :
        #stack data
        STACK_DATA = np.vstack((STACK_DATA ,DATA))
    else: 
        # if DATA.isna().sum().sum() > 0:
        #     print("Data contains NaNs. Please handle them before scaling.")
        STACK_DATA = np.empty((0,) + (32,9) )
        X = DATA[:, :, :3]  # เลือกข้อมูลแถวแรกถึงแถวที่ 3 สำหรับ X
        Y = DATA[:, :, 3:]  # เลือกข้อมูลแถวที่ 3 เป็นต้นไปสำหรับ Y
        x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
        random.shuffle(PartitionDate)
        tf.keras.backend.clear_session() 
        history = model.fit(x_train , y_train, epochs=CONFIG['epochs'], batch_size=BATCH_SIZE, validation_data=(x_val, y_val) )
        #history = model.fit(x_train , y_train, epochs=5, batch_size=BATCH_SIZE )
        if  np.isnan(  np.average( history.history['kl_loss'] )  ) or np.isnan(  np.average( history.history['val_kl_loss'] )  ):
            STOP_MODEL = True 
            print(x_train)
            print("---")
            print(x_val)
            print("=============")
        if WANDB_LOG :
            LogKeys = history.history.keys()
            LogVal={}
            for k in LogKeys:  
                LogVal[k] = np.average(  history.history[k] )
            wandb.log(LogVal, commit=True)
        
    if STOP_MODEL :
        break
    
            
    model.encoder.save(model_path+f'/'+f'encoder.keras') 
    model.decoder.save(model_path+f'/'+f'decoder.keras') 
if WANDB_LOG : wandb.finish()

In [ ]:
`====================================================

In [ ]:
======================== predict =========================

In [5]:
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
for partdate in PartitionDate[:1] :
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    if CONFIG['transform'] :
        DATA_TF = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
        DATA_TF = DATA_TF.reshape(data_shape)

In [87]:
X = DATA_TF[:, :, :3][:1]
# X data
df = pd.DataFrame(
    DATA[:, :, :3][:1].reshape(32, 3), 
    columns=SCALER_COL[:3])
#print(df)

In [15]:
z_mean, log_var = model.encoder(X) 
z = Sampling()([z_mean, log_var])
decode_data = model.decoder(z)

In [18]:
======================= _compute_loss ==============================

In [105]:
#generated_data = [c_bid, c_ask, c_volume, p_bid, p_ask, p_volume]
colList = ["c_bid", "c_ask", "c_volume", "p_bid", "p_ask", "p_volume"]
generated_data = decode_data[3:]
z_mean    = z_mean
z_log_var = log_var
Y_real    = DATA[:, :, 3:][:1]

In [128]:
for  col,genData in zip(colList,generated_data):
    print( colList.index(col),col )

array([1.188e+01, 1.216e+01, 1.100e+01, 9.000e-02, 1.000e-01, 6.060e+02])

In [142]:
subtract_genData = genData - tf.cast(tf.expand_dims(Y_real[:, :, colList.index(col)], axis=-1)
        , tf.float32) 

In [154]:
reconstruction_values_total = []
reconstruction_values_total.append( tf.reduce_mean( tf.square(subtract_genData)   ) )

In [150]:
log_var = tf.clip_by_value(log_var, -1.0, 1.0)
kl_loss = -0.5 * tf.reduce_sum(1 + log_var - tf.square(z_mean) - tf.exp(log_var), axis=-1)

In [155]:
tf.reduce_mean(reconstruction_values_total + kl_loss)

<tf.Tensor: shape=(), dtype=float32, numpy=19854.121>

In [ ]:
======================= inverse_transform ========================

In [60]:
#add 0
decode_data = [tf.zeros([1, 32, 1])]*3 + decode_data

In [78]:
invert_decode = Scaler.inverse_transform(
    np.array([d.numpy().reshape(-1) for d in decode_data]).transpose()
    ) 

In [85]:
pd.DataFrame(
    invert_decode[:,3:], 
    columns=SCALER_COL[3:])

,C_BID,C_ASK,C_VOLUME,P_BID,P_ASK,P_VOLUME
0,68.616890,112.805359,57.281818,94.043495,35.460060,87.271248
1,144.180679,75.689438,588.284241,51.924221,61.831406,87.271248
2,99.913605,70.390205,57.281818,34.165512,45.765572,220.349365
3,68.616890,85.750114,57.281818,38.844898,52.346127,87.271248
4,68.616890,114.237602,563.369873,34.165512,97.664665,443.883575
5,72.891563,70.390205,303.591736,34.165512,35.460060,149.589447
6,73.816498,70.390205,703.170227,34.165512,143.152863,87.271248
7,97.970665,100.458321,57.281818,34.165512,94.341377,87.271248
8,160.380630,110.638863,57.281818,66.755714,35.460060,553.159912
9,76.275078,127.448189,57.281818,34.165512,35.856621,87.271248


In [86]:
=====================================================================

SyntaxError: invalid syntax (3079178945.py, line 1)